In [ ]:
# 读取已下载的数据到 DataFrame
import os
from pathlib import Path

from freqtrade.data.history import load_pair_history


def _find_project_root(start: Path | None = None) -> Path:
    start = start or Path.cwd()
    for p in (start, *start.parents):
        if (p / "config.json").is_file():
            return p
    raise FileNotFoundError("未找到项目根目录：从当前目录向上未发现 config.json")


PROJECT_ROOT = _find_project_root()
# 统一工作目录到项目根目录，避免相对路径（例如 user_data_dir="."）随 Notebook 运行目录漂移。
os.chdir(PROJECT_ROOT)

df = load_pair_history(datadir=Path("data/okx"), timeframe="1h", pair="BTC/USDT")
if df.empty:
    raise ValueError(
        f"未在 {Path('data/okx').resolve()} 读取到 BTC/USDT 1h 历史数据，请检查数据是否已下载/路径是否正确。"
    )
df.tail()


In [ ]:
# 分类模型
import numpy as np


def classification_tc(df):
    # 开盘价-收盘价，结果大于为阴线，小于为阳线
    df["Open-Close"] = df["open"] - df["close"]
    # 最高价-最低价，衡量这根k线的长度，价格波动大小
    df["High-Low"] = df["high"] - df["low"]
    # 目标变量，下一根k线是否涨
    df["target"] = np.where(df["close"].shift(-1) > df["close"], 1, -1)
    df = df.dropna()
    x = df[["Open-Close", "High-Low"]]
    y = df["target"]
    return x, y

In [ ]:
# 拆分训练集与验证集
from sklearn.model_selection import train_test_split

# 使用classification_tc函数生成数据集的特征与目标
x, y = classification_tc(df)
# 将数据集拆分为训练集与验证集
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)
print(x_train, "\n", x_test)

In [ ]:
# 训练模型
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=95)
# 使用KNN拟合训练集
knn_clf.fit(x_train, y_train)
# 输出模型在训练集中的准确率
print(knn_clf.score(x_train, y_train))
# 输出模型在验证集中的准确率
print(knn_clf.score(x_test, y_test))

In [ ]:
# 模型持久化保存
from pathlib import Path

import joblib

model_path = Path("models/knn_btc_usdt_1h.pkl")
model_path.parent.mkdir(parents=True, exist_ok=True)
# 保存模型到项目根目录的 models/ 下，便于策略直接加载。
joblib.dump(knn_clf, model_path)

In [ ]:
# 信号可视化
# 买卖点绘图
import os
from pathlib import Path

from freqtrade.configuration import Configuration
from freqtrade.data.dataprovider import DataProvider
from freqtrade.plot.plotting import generate_candlestick_graph
from freqtrade.resolvers import StrategyResolver

pair = "BTC/USDT"

try:
    PROJECT_ROOT  # type: ignore
except NameError:

    def _find_project_root(start: Path | None = None) -> Path:
        start = start or Path.cwd()
        for p in (start, *start.parents):
            if (p / "config.json").is_file():
                return p
        raise FileNotFoundError("未找到项目根目录：从当前目录向上未发现 config.json")

    PROJECT_ROOT = _find_project_root()
    os.chdir(PROJECT_ROOT)

config = Configuration.from_files([str(PROJECT_ROOT / "config.json")])  # type: ignore

config["strategy"] = "KNNStrategy"

strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)

strategy.ft_bot_start()

if "df" not in globals():
    raise ValueError("未找到 df 变量：请先运行最上面的“读取数据”单元。")
if df is None or df.empty:
    raise ValueError(
        "df 为空，无法生成信号；请检查数据是否已成功加载（data/okx 下是否有数据）。"
    )

df = strategy.analyze_ticker(df, {"pair": pair})

# 绘图
graph = generate_candlestick_graph(data=df, pair=pair)
graph.show()

In [ ]:
# 回测
!freqtrade backtesting \
    --config ./config.json \
    --userdir ./ \
    --strategy KNNStrategy \
    --export trades \
    --timeframe 1h
# --export trades 的作用是：把回测过程中产生的“每一笔交易记录”导出保存到结果文件里（包含进场/出场时间、价格、数量、盈亏等），方便后续用来画图或做分析；结果文件默认会写到 backtest_results/（也可以用 --backtest-directory 改目录）。

Loading KNN model from: models\knn_btc_usdt_1h.pkl
Result for strategy KNNStrategy
                                             BACKTESTING REPORT                                              
┌──────────┬────────┬──────────────┬─────────────────┬──────────────┬──────────────┬────────────────────────┐
│     Pair │ Trades │ Avg Profit % │ Tot Profit USDT │ Tot Profit % │ Avg Duration │  Win  Draw  Loss  Win% │
├──────────┼────────┼──────────────┼─────────────────┼──────────────┼──────────────┼────────────────────────┤
│ BTC/USDT │    179 │        -0.24 │        -131.081 │       -13.11 │      2:19:00 │   30     0   149  16.8 │
│    TOTAL │    179 │        -0.24 │        -131.081 │       -13.11 │      2:19:00 │   30     0   149  16.8 │
└──────────┴────────┴──────────────┴─────────────────┴──────────────┴──────────────┴────────────────────────┘
                                           LEFT OPEN TRADES REPORT                                           
┌──────────┬────────┬──────────────┬─

2026-01-07 00:01:37,583 - freqtrade - INFO - freqtrade 2026.1-dev-0acd1d8
2026-01-07 00:01:38,875 - freqtrade.configuration.load_config - INFO - Using config: ./config.json ...
2026-01-07 00:01:38,877 - freqtrade.loggers - INFO - Enabling colorized output.
2026-01-07 00:01:38,878 - freqtrade.loggers - INFO - Logfile configured
2026-01-07 00:01:38,878 - freqtrade.loggers - INFO - Verbosity set to 0
2026-01-07 00:01:38,878 - freqtrade.configuration.configuration - INFO - Parameter -i/--timeframe detected ... Using timeframe: 1h ...
2026-01-07 00:01:38,878 - freqtrade.configuration.configuration - INFO - Using max_open_trades: 3 ...
2026-01-07 00:01:38,879 - freqtrade.configuration.configuration - INFO - Using user-data directory: . ...
2026-01-07 00:01:38,879 - freqtrade.configuration.configuration - INFO - Using data directory: data\okx ...
2026-01-07 00:01:38,879 - freqtrade.configuration.configuration - INFO - Parameter --export detected: trades ...
2026-01-07 00:01:38,880 - freqtrade

In [ ]:
# 回测结果图表可视化
import pandas as pd
import plotly.express as px
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats

strategy = "KNNStrategy"

backtest_dir = Path(config["user_data_dir"], "backtest_results")

stats = load_backtest_stats(backtest_dir)
strategy_stats = stats["strategy"][strategy]

df = pd.DataFrame(columns=["dates", "equity"], data=strategy_stats["daily_profit"])
df["equity_daily"] = df["equity"].cumsum()

fig = px.line(df, x="dates", y="equity_daily")
fig.show()

2026-01-07 00:06:31,613 - freqtrade.data.btanalysis.bt_fileutils - INFO - Loading backtest result from backtest_results\backtest-result-2026-01-07_00-01-40.zip

In [26]:
# 实际交易点位
trades = load_backtest_data(backtest_dir)

pair = "BTC/USDT"

df = load_pair_history(datadir=Path("./data/okx"), timeframe="1h", pair="BTC/USDT")

graph = generate_candlestick_graph(data=df, pair=pair, trades=trades)  # 显示图表
graph.show()

2026-01-07 00:10:03,395 - freqtrade.data.btanalysis.bt_fileutils - INFO - Loading backtest result from backtest_results\backtest-result-2026-01-07_00-01-40.zip

2026-01-07 00:10:03,455 - freqtrade.plot.plotting - INFO - Indicator "sma" ignored. Reason: This indicator is not found in your strategy.

2026-01-07 00:10:03,457 - freqtrade.plot.plotting - INFO - Indicator "ema3" ignored. Reason: This indicator is not found in your strategy.

2026-01-07 00:10:03,457 - freqtrade.plot.plotting - INFO - Indicator "ema5" ignored. Reason: This indicator is not found in your strategy.

2026-01-07 00:10:03,481 - freqtrade.plot.plotting - INFO - Indicator "macd" ignored. Reason: This indicator is not found in your strategy.

2026-01-07 00:10:03,484 - freqtrade.plot.plotting - INFO - Indicator "macdsignal" ignored. Reason: This indicator is not found in your strategy.